# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,marechal floriano,-20.4128,-40.6831,20.14,98,21,1.42,BR,11/05/2024
1,1,baker city,44.7749,-117.8344,14.25,83,0,1.54,US,11/05/2024
2,2,adamstown,-25.0660,-130.1015,21.81,79,99,5.63,PN,11/05/2024
3,3,grytviken,-54.2811,-36.5092,-3.06,83,67,4.83,GS,11/05/2024
4,4,tolanaro,-25.0319,46.9987,26.86,61,0,6.57,MG,11/05/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points('Lng', 'Lat', 
    geo=True, alpha=0.6,
    xlim=(-180, 180), ylim=(-90, 90), tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    color='City',
    hover_cols=['Max Temp'])

# Display the map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

My ideal weather condition
- Temperature no lower than 24, and no higher than 30 degrees Celsius
- Humidty higher than 60 %
- Wind speed no more than 4.5 m/s

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] > 24) & (city_data_df["Max Temp"] < 30)]
city_data_df = city_data_df[city_data_df['Humidity'] > 60]
city_data_df = city_data_df[city_data_df['Wind Speed'] < 4.5]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,lambarene,-0.7001,10.2406,28.90,75,91,1.39,GA,11/05/2024
35,35,el porvenir,13.7617,-87.3458,24.63,88,100,0.39,HN,11/05/2024
42,42,cabinda,-5.5500,12.2000,27.71,84,100,2.28,AO,11/05/2024
43,43,acarau,-2.8856,-40.1200,24.72,89,97,3.84,BR,11/05/2024
46,46,avarua,-21.2078,-159.7750,25.03,78,20,2.06,CK,11/05/2024
63,63,mahibadhoo,3.7833,72.9667,29.59,74,98,4.01,MV,11/05/2024
73,73,lae,-6.7221,146.9847,26.61,85,81,2.34,PG,11/05/2024
90,90,bubaque,11.2833,-15.8333,26.51,76,11,4.13,GW,11/05/2024
109,109,mahina,-17.5065,-149.4890,24.87,94,75,2.57,PF,11/05/2024
120,120,buala,-8.1450,159.5921,27.12,84,100,2.29,SB,11/05/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
417,freetown,SL,8.4840,-13.2299,89,
394,fougamou,GA,-1.2154,10.5838,76,
416,robertsport,LR,6.7533,-11.3686,77,
172,fale old settlement,TK,-9.3852,-171.2468,74,
484,manokwari,ID,-0.8667,134.0833,85,
530,palikir - national government center,FM,6.9248,158.1611,89,
450,tucuma,BR,-0.1833,-50.0500,95,
212,san luis de la loma,MX,17.2706,-100.8939,72,
371,meulaboh,ID,4.1363,96.1285,74,
550,da nang,VN,16.0678,108.2208,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Latitude and longitude of each city
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    url = f"{base_url}?filter={params['filter']}&bias={params['bias']}&apiKey={params['apiKey']}&categories=accommodation.hotel"
    name_address = requests.get(url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lambarene - nearest hotel: Motel Obama
el porvenir - nearest hotel: No hotel found
cabinda - nearest hotel: Hotel Mayombe
acarau - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
mahibadhoo - nearest hotel: No hotel found
lae - nearest hotel: Huon Gulf Hotel
bubaque - nearest hotel: Chez Julio
mahina - nearest hotel: Motu Martin
buala - nearest hotel: No hotel found
la passe - nearest hotel: L'Estuaire
ambon - nearest hotel: Hotel Hero
bondoukou - nearest hotel: Hotel Akakro
namibe - nearest hotel: Nelsal Pensao
uturoa - nearest hotel: Hawaiki Nui hotel
moyamba - nearest hotel: Samies
brisas de zicatela - nearest hotel: Casa de Olas
fale old settlement - nearest hotel: No hotel found
malango - nearest hotel: Madarana
bikenibeu village - nearest hotel: Otintaai Hotel
carupano - nearest hotel: Hotel El Yunque
port isabel - nearest hotel: Queen Isabel Inn
hengchun - nearest hotel: 海的顏色精品旅館
san luis de la loma - nearest hotel: No hotel found
kerema 

,City,Country,Lat,Lng,Humidity,Hotel Name
29,lambarene,GA,-0.7001,10.2406,75,Motel Obama
35,el porvenir,HN,13.7617,-87.3458,88,No hotel found
42,cabinda,AO,-5.5500,12.2000,84,Hotel Mayombe
43,acarau,BR,-2.8856,-40.1200,89,No hotel found
46,avarua,CK,-21.2078,-159.7750,78,Paradise Inn
...,...,...,...,...,...,...
530,palikir - national government center,FM,6.9248,158.1611,89,Mangrove Bay Hotel
543,bonthe,SL,7.5264,-12.5050,72,No hotel found
548,katsuren-haebaru,JP,26.3369,127.8719,68,No hotel found
550,da nang,VN,16.0678,108.2208,74,Thanh Thanh Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points('Lng', 'Lat', 
    geo=True, alpha=0.6,
    xlim=(-180, 180), ylim=(-90, 90), tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    color='City',
    hover_cols=['Country', 'Hotel Name'])

# Display the map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)